In [5]:
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

train, valid = tfds.load(
    "div2k/bicubic_x4", 
    split=["train","validation"],
    as_supervised=True
)
def preprocessing(lr, hr):
    hr = tf.cast(hr, tf.float32) /255.
        
    # 이미지의 크기가 크므로 (96,96,3) 크기로 임의 영역을 잘라내어 사용합니다.
    hr_patch = tf.image.random_crop(hr, size=[96,96,3])
        
    # 잘라낸 고해상도 이미지의 가로, 세로 픽셀 수를 1/4배로 줄입니다
    # 이렇게 만든 저해상도 이미지를 SRGAN의 입력으로 사용합니다.
    lr_patch = tf.image.resize(hr_patch, [96//4, 96//4], "bicubic")
    return lr_patch, hr_patch

train = train.map(preprocessing).shuffle(buffer_size=10).batch(8)
valid = valid.map(preprocessing).batch(8)
print("✅")

✅


In [2]:
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

train, valid = tfds.load(
    "div2k/bicubic_x4", 
    split=["train","validation"],
    as_supervised=True
)
def preprocessing(lr, hr):
    hr = tf.cast(hr, tf.float32) /255.
        
    # 이미지의 크기가 크므로 (96,96,3) 크기로 임의 영역을 잘라내어 사용합니다.
    hr_patch = tf.image.random_crop(hr, size=[96,96,3])
        
    # 잘라낸 고해상도 이미지의 가로, 세로 픽셀 수를 1/4배로 줄입니다
    # 이렇게 만든 저해상도 이미지를 SRGAN의 입력으로 사용합니다.
    lr_patch = tf.image.resize(hr_patch, [96//4, 96//4], "bicubic")
    return lr_patch, hr_patch

train = train.map(preprocessing).shuffle(buffer_size=10).repeat().batch(8)
valid = valid.map(preprocessing).repeat().batch(8)
print("✅")

✅


In [3]:
from tensorflow.keras import Input, Model, layers

# 그림의 파란색 블록을 정의합니다.
def gene_base_block(x):
    out = layers.Conv2D(64, 3, 1, "same")(x)
    out = layers.BatchNormalization()(out)
    out = layers.PReLU(shared_axes=[1,2])(out)
    out = layers.Conv2D(64, 3, 1, "same")(out)
    out = layers.BatchNormalization()(out)
    return layers.Add()([x, out])

# 그림의 뒤쪽 연두색 블록을 정의합니다.
def upsample_block(x):
    out = layers.Conv2D(256, 3, 1, "same")(x)
    # 그림의 PixelShuffler 라고 쓰여진 부분을 아래와 같이 구현합니다.
    out = layers.Lambda(lambda x: tf.nn.depth_to_space(x, 2))(out)
    return layers.PReLU(shared_axes=[1,2])(out)
    
# 전체 Generator를 정의합니다.
def get_generator(input_shape=(None, None, 3)):
    inputs = Input(input_shape)
    
    out = layers.Conv2D(64, 9, 1, "same")(inputs)
    out = residual = layers.PReLU(shared_axes=[1,2])(out)
    
    for _ in range(5):
        out = gene_base_block(out)
    
    out = layers.Conv2D(64, 3, 1, "same")(out)
    out = layers.BatchNormalization()(out)
    out = layers.Add()([residual, out])
    
    for _ in range(2):
        out = upsample_block(out)
        
    out = layers.Conv2D(3, 9, 1, "same", activation="tanh")(out)
    return Model(inputs, out)

print("✅")

✅


In [4]:
from tensorflow.keras import losses, metrics, optimizers

generator = get_generator()

# 사용할 loss function 및 optimizer 를 정의합니다.
mse = losses.MeanSquaredError()
gene_opt = optimizers.Adam()

@tf.function
def get_content_loss(hr_real, hr_fake):
    return mse(hr_real, hr_fake)


@tf.function
def step(lr, hr_real):
    with tf.GradientTape() as gene_tape, tf.GradientTape() as disc_tape:
        hr_fake = generator(lr, training=True)
        
        perceptual_loss = get_content_loss(hr_real, hr_fake)
        
    gene_gradient = gene_tape.gradient(perceptual_loss, generator.trainable_variables)
    
    gene_opt.apply_gradients(zip(gene_gradient, generator.trainable_variables))
    return perceptual_loss


In [ ]:
gene_losses = metrics.Mean()

for epoch in range(1, 10):
    for i, (lr, hr) in enumerate(train):
        g_loss = step(lr, hr)
        
        gene_losses.update_state(g_loss)
        
        # 10회 반복마다 loss를 출력합니다.
        if (i+1) % 10 == 0:
            print(f"EPOCH[{epoch}] - STEP[{i+1}] \nGenerator_loss:{gene_losses.result():.4f}", end="\n\n")
                    
    gene_losses.reset_states()

EPOCH[1] - STEP[10] 
Generator_loss:0.0909

EPOCH[1] - STEP[20] 
Generator_loss:0.0655

EPOCH[1] - STEP[30] 
Generator_loss:0.0529

EPOCH[1] - STEP[40] 
Generator_loss:0.0457

EPOCH[1] - STEP[50] 
Generator_loss:0.0405

EPOCH[1] - STEP[60] 
Generator_loss:0.0369

EPOCH[1] - STEP[70] 
Generator_loss:0.0341

EPOCH[1] - STEP[80] 
Generator_loss:0.0319

EPOCH[1] - STEP[90] 
Generator_loss:0.0300

EPOCH[1] - STEP[100] 
Generator_loss:0.0283

EPOCH[1] - STEP[110] 
Generator_loss:0.0273

EPOCH[1] - STEP[120] 
Generator_loss:0.0264

EPOCH[1] - STEP[130] 
Generator_loss:0.0254

EPOCH[1] - STEP[140] 
Generator_loss:0.0245

EPOCH[1] - STEP[150] 
Generator_loss:0.0238

EPOCH[1] - STEP[160] 
Generator_loss:0.0230

EPOCH[1] - STEP[170] 
Generator_loss:0.0224

EPOCH[1] - STEP[180] 
Generator_loss:0.0217

EPOCH[1] - STEP[190] 
Generator_loss:0.0212

EPOCH[1] - STEP[200] 
Generator_loss:0.0206

EPOCH[1] - STEP[210] 
Generator_loss:0.0202

EPOCH[1] - STEP[220] 
Generator_loss:0.0197

EPOCH[1] - STEP[230

EPOCH[1] - STEP[1820] 
Generator_loss:0.0084

EPOCH[1] - STEP[1830] 
Generator_loss:0.0083

EPOCH[1] - STEP[1840] 
Generator_loss:0.0083

EPOCH[1] - STEP[1850] 
Generator_loss:0.0083

EPOCH[1] - STEP[1860] 
Generator_loss:0.0083

EPOCH[1] - STEP[1870] 
Generator_loss:0.0083

EPOCH[1] - STEP[1880] 
Generator_loss:0.0083

EPOCH[1] - STEP[1890] 
Generator_loss:0.0083

EPOCH[1] - STEP[1900] 
Generator_loss:0.0082

EPOCH[1] - STEP[1910] 
Generator_loss:0.0082

EPOCH[1] - STEP[1920] 
Generator_loss:0.0082

EPOCH[1] - STEP[1930] 
Generator_loss:0.0082

EPOCH[1] - STEP[1940] 
Generator_loss:0.0082

EPOCH[1] - STEP[1950] 
Generator_loss:0.0082

EPOCH[1] - STEP[1960] 
Generator_loss:0.0082

EPOCH[1] - STEP[1970] 
Generator_loss:0.0082

EPOCH[1] - STEP[1980] 
Generator_loss:0.0082

EPOCH[1] - STEP[1990] 
Generator_loss:0.0082

EPOCH[1] - STEP[2000] 
Generator_loss:0.0081

EPOCH[1] - STEP[2010] 
Generator_loss:0.0081

EPOCH[1] - STEP[2020] 
Generator_loss:0.0081

EPOCH[1] - STEP[2030] 
Generator_l

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf

def apply_srgan(image):
    image = tf.cast(image[np.newaxis, ...], tf.float32)
    sr = generator.predict(image/255)
    sr = sr * 255
    sr = tf.clip_by_value(sr, 0, 255)
    sr = tf.round(sr)
    sr = tf.cast(sr, tf.uint8)
    return np.array(sr)[0]

_, valid = tfds.load(
    "div2k/bicubic_x4", 
    split=["train", "validation"],
    as_supervised=True
)

for i, (lr, hr) in enumerate(valid):
    if i == 6: break

srgan_hr = apply_srgan(lr)
print("✅")

In [ ]:
# 이미지의 특정 부분을 잘라내는 함수를 정의합니다.
def crop(image, left_top, x=200, y=200):
    return image[left_top[0]:(left_top[0]+x), left_top[1]:(left_top[1]+y), :]

hr, lr = np.array(hr), np.array(lr)
bicubic_hr = cv2.resize(
    lr, 
    dsize=(hr.shape[1], hr.shape[0]), # 고해상도 이미지 크기로 설정
    interpolation=cv2.INTER_CUBIC # bicubic 설정
)

In [ ]:
# 자세히 시각화 하기 위해 3개 영역을 잘라냅니다.
# 아래는 잘라낸 부분의 좌상단 좌표 3개 입니다.
left_tops = [(400,500), (300,1200), (0,1000)]

images = []
for left_top in left_tops:
    img1 = crop(bicubic_hr, left_top, 200, 200)
    img2 = crop(srgan_hr , left_top, 200, 200)
    img3 = crop(hr, left_top, 200, 200)
    images.extend([img1, img2, img3])

labels = ["Bicubic", "SRGAN_MSE", "HR"] * 3

plt.figure(figsize=(18,18))
for i in range(9):
    plt.subplot(3,3,i+1) 
    plt.imshow(images[i])
    plt.title(labels[i], fontsize=30)